<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/PAPER_GDRNet_SEED-ALLDATASET-MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input, LeakyReLU
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
from math import sqrt

# ====================================================
# Load data
# ====================================================

##MetroPM
y = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy')
x = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data2.npy')

##AQ
y = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated-data-true-window2.npy')
x = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated-data2.npy')

##ELECTRICITY
x=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data2.npy')
y=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data-true-window2.npy')

from sklearn.preprocessing import PowerTransformer
transformer = StandardScaler()
y_transformed = transformer.fit_transform(y.reshape(-1,1)).flatten()
x_train, x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.1, random_state = 42)

# ======================= Model Builder ==========================
def build_model(input_dim):
    model = Sequential()
    model.add(Dense(units=64, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(units=32)); model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(units=16)); model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(units=8));  model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(units=1))
    optim = keras.optimizers.Adam(learning_rate=0.001, clipnorm=1)
    model.compile(loss='mean_squared_error', optimizer=optim)
    return model

# ======================= Robustness Runs ========================
seeds = [42, 123, 999, 2024, 7]
rmse_scores = []

for seed in seeds:
    print(f"\n===== Running seed {seed} =====")

    # Set seeds for reproducibility
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # Train-test split
    x_train, x_test, y_train, y_test = train_test_split(
        x, y_transformed, test_size=0.1, random_state=seed
    )

    # Build and train
    model = build_model(x.shape[1])

    cp_callback = ModelCheckpoint(
        filepath=f"/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/ckp_seed{seed}.weights.h5",
        monitor='loss', save_best_only=True, save_weights_only=True
    )
    es = EarlyStopping(patience=20, verbose=1, min_delta=1e-4, monitor='loss',
                       mode='min', restore_best_weights=True)

    history = model.fit(
        x_train, y_train,
        epochs=50, batch_size=50,
        validation_split=0.2,
        callbacks=[cp_callback, es],
        verbose=1
    )

    # Evaluation
    y_pred = model.predict(x_test)
    y_test_inv = transformer.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_inv = transformer.inverse_transform(y_pred).flatten()
    rmse = math.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    rmse_scores.append(rmse)

    print(f"Seed {seed} RMSE: {rmse:.4f}")

# ======================= Summary ================================
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)
print("\n=== Robustness Report (GDRNet MLP) ===")
print(f"Seeds tested: {seeds}")
print(f"RMSE mean ± std: {mean_rmse:.4f} ± {std_rmse:.4f}")



===== Running seed 42 =====
Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1963/1963 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.9144 - val_loss: 0.8309
Epoch 2/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8293 - val_loss: 0.7956
Epoch 3/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8038 - val_loss: 0.7856
Epoch 4/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7910 - val_loss: 0.7844
Epoch 5/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7810 - val_loss: 0.7776
Epoch 6/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7735 - val_loss: 0.7802
Epoch 7/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7678 - val_loss: 0.7744
Epoch 8/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7626 - val_loss: 0.7730
Epoch 9/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7583 - val_loss: 0.7803
Epoch 10/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7548 - val_loss: 0.7724
Epoch 11/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7533 - val_loss: 0.7732
Epoch 12/50
1963/1963 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1963/1963 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.9188 - val_loss: 0.8244
Epoch 2/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8238 - val_loss: 0.8035
Epoch 3/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8012 - val_loss: 0.8154
Epoch 4/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7898 - val_loss: 0.8070
Epoch 5/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7805 - val_loss: 0.8013
Epoch 6/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7748 - val_loss: 0.8062
Epoch 7/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7698 - val_loss: 0.8228
Epoch 8/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7659 - val_loss: 0.7983
Epoch 9/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7607 - val_loss: 0.7980
Epoch 10/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7559 - val_loss: 0.7942
Epoch 11/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7533 - val_loss: 0.7821
Epoch 12/50
1963/1963 ━━━━━━━━━━━━━━━━━━━

...

1963/1963 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.9193 - val_loss: 0.8211
Epoch 2/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8233 - val_loss: 0.7950
Epoch 3/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8003 - val_loss: 0.7850
Epoch 4/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7890 - val_loss: 0.7745
Epoch 5/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7801 - val_loss: 0.7686
Epoch 6/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7737 - val_loss: 0.7651
Epoch 7/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7692 - val_loss: 0.7568
Epoch 8/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7643 - val_loss: 0.7622
Epoch 9/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7612 - val_loss: 0.7546
Epoch 10/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7574 - val_loss: 0.7516
Epoch 11/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.7538 - val_loss: 0.7489
Epoch 12/50
1963/1963 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1963/1963 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.9279 - val_loss: 0.8158
Epoch 2/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8253 - val_loss: 0.8046
Epoch 3/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8036 - val_loss: 0.8021
Epoch 4/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7916 - val_loss: 0.7889
Epoch 5/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7834 - val_loss: 0.7801
Epoch 6/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7764 - val_loss: 0.7848
Epoch 7/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7709 - val_loss: 0.7856
Epoch 8/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7660 - val_loss: 0.7765
Epoch 9/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7607 - val_loss: 0.7666
Epoch 10/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7578 - val_loss: 0.7615
Epoch 11/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7540 - val_loss: 0.7614
Epoch 12/50
1963/1963 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1963/1963 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.9252 - val_loss: 0.8265
Epoch 2/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8290 - val_loss: 0.8042
Epoch 3/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8039 - val_loss: 0.7834
Epoch 4/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7898 - val_loss: 0.7769
Epoch 5/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7793 - val_loss: 0.7711
Epoch 6/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7721 - val_loss: 0.7692
Epoch 7/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7661 - val_loss: 0.7645
Epoch 8/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7609 - val_loss: 0.7583
Epoch 9/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7562 - val_loss: 0.7559
Epoch 10/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7534 - val_loss: 0.7513
Epoch 11/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7503 - val_loss: 0.7502
Epoch 12/50
1963/1963 ━━━━━━━━━━━━━━━━━━━